In [22]:
import os
import boto3
import mysql.connector
import csv
import time
import io
import pandas as pd
from boto3.session import Session
from dateutil.parser import parse

def readallfiles (x,y,z,q,x1,y1,z1,q1):
    # x,y,z,q refer to the parameters of the aws s3 bucket connection
    #x1,y1,z1,q1 refer to the parameters of the aws rds database connection
    a,c=connectaws(x,y,z,q)
    _conn= ConnectionDatabase(x1,y1,z1,q1)
    datalist=[]
    for key in a:
        date1 = key.split("_")
        a1=str(date1[0])
        b1=parse(a1)
        obj = c.get_object(Bucket= q , Key = key)
        df = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8')
        df1=df.values.tolist()
        df2=df1[0]
        datalist.append(df2)
        c.delete_object(Bucket=q, Key=key)
        counter = 0
        for row in datalist:
        # row[0]=Username,row[1]=site id,row[2]=X,row[3]=Y,row[4]=water level
        #row[5]=stream gage, row[6]=groundwater, row[7]=soil moisture,row[8]=temperature
        #row[9]=precipitation, row[10]=linkageID
            daterecord={}
            daterecord["Date"]=b1
            daterecord["username"] = row[0]
            daterecord["site_id"] = row[1]
            daterecord["latitude"] = row[2]
            daterecord["longitude"] = row[3]
            daterecord["water_level"] = row[4]
            daterecord["stream_gage"] = row[5]
            daterecord["groundwater_level"] = row[6]
            daterecord["soil_moisture"] = row[7]
            daterecord["temperature"] = row[8]
            daterecord["precipitation"] = row[9]
            daterecord["linkage_id"] = row[10]
        if insertdaterecord(_conn,daterecord):
            counter+=1
        if counter != 0:
            _conn.commitData()
        datalist=[]
        print("Insert the data flow: %d" %counter)
    _conn.closeConn()

def insertdaterecord(_conn,daterecord):
    
    _sql = (
                "INSERT INTO date_record(Date,username,site_id,latitude,longitude,water_level,stream_gage,groundwater_level,soil_moisture,temperature,precipitation,linkage_id)"
                "values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)")
    _params = (daterecord["Date"],daterecord["username"], daterecord["site_id"],
                    daterecord["latitude"],
                    daterecord["longitude"],
                    daterecord["water_level"],
                    daterecord["stream_gage"],
                    daterecord["groundwater_level"],
                    daterecord["soil_moisture"],
                    daterecord["temperature"],
                    daterecord["precipitation"],
                    daterecord["linkage_id"])
    return _conn.mysql_exe_sql(_sql,_params)

# create the database connection class
class ConnectionDatabase(object):
    # connect python to mysql database
    def __init__(self,host, user_name, passwd, db, char='utf8'):
        self.ip = host
        self.username = user_name
        self.passwd = passwd
        self.mysqldb = db
        self.char = char

        self.mysql_db = mysql.connector.connect(
            host=self.ip,
            user=self.username,
            password=self.passwd,
            database=self.mysqldb,
            charset=self.char)
    # query the database（mysql）
    def mysql_findList(self, sql):
        cursor = self.mysql_db.cursor()
        MySQL_sql = sql
        results = None
        if not cursor:
            raise (NameError,"fail to connect the database")
        try:
            cursor.execute(MySQL_sql)
            # get the results
            results = cursor.fetchall()
        except Exception as e:
            print(e)
            self.mysql_db.close()
        if results:
            return results
        else:
            return None

    # data change（mysql）
    def mysql_exe_sql(self, sql, params):
        cursor = self.mysql_db.cursor()
        MySQL_sql = sql
        result = 0
        if not cursor:
            raise (NameError,"fail to connect the database")
        try:
            cursor.execute(MySQL_sql, params)
            result = cursor.rowcount
        except Exception as e:
            print(e)
            self.mysql_db.rollback()
            self.mysql_db.close()

        return result>0
    #commit the data
    def commitData(self):
        try:
            self.mysql_db.commit()
        except Exception as e:
            print(e)
    #close the connection with the database
    def closeConn(self):
        if self.mysql_db:
            self.mysql_db.close()
            
def connectaws(REGION,ACCESS_KEY_ID,SECRET_ACCESS_KEY,BUCKET_NAME):
#REGION = ""
#ACCESS_KEY_ID = ""
#SECRET_ACCESS_KEY = ""
#BUCKET_NAME = ""
    session = Session(
                      aws_access_key_id=ACCESS_KEY_ID,
                      aws_secret_access_key=SECRET_ACCESS_KEY)
    s3c = boto3.client(
            's3', 
            region_name = REGION,
            aws_access_key_id = ACCESS_KEY_ID,
            aws_secret_access_key = SECRET_ACCESS_KEY
        )
    s3 = session.resource('s3')
    your_bucket = s3.Bucket(BUCKET_NAME)
    keys=[]
    datalist=[]
    for s3_file in your_bucket.objects.all():
        keys.append(s3_file.key)

    return keys,s3c

readallfiles()

''''
while True:
  time_now = time.strftime("%H:%M:%S", time.localtime())
  if time_now == "21:43:01":
    dir = "C://Users//shuol//Desktop//test"
    readallfiles(dir)
    subject = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "sent the test"
    print(subject)
    time.sleep(5)
'''

'\'\nwhile True:\n  time_now = time.strftime("%H:%M:%S", time.localtime())\n  if time_now == "21:43:01":\n    dir = "C://Users//shuol//Desktop//test"\n    readallfiles(dir)\n    subject = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "sent the test"\n    print(subject)\n    time.sleep(5)\n'